In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep

In [22]:
url = 'https://www.thegioididong.com/dtdd'

driver = webdriver.Chrome()
driver.get(url)

In [23]:
accessory_elm = driver.find_element(By.XPATH, '/html/body/header/div[2]/div/ul/li[3]')
accessory_elm.click()

In [24]:
charger_elm = driver.find_element(By.XPATH, '/html/body/div[7]/div[5]/section[1]/div/div[2]/a')
charger_elm.click()

In [25]:
best_seller_elm = driver.find_element(By.XPATH, '/html/body/div[7]/section[4]/div[3]/div/div[2]/div[1]/div[2]/p[2]/a')
best_seller_elm.click()

In [26]:
show_more_elm = driver.find_element(By.XPATH, '//*[@id="categoryPage"]/div[1]/div[2]/a/strong')
show_more_elm


<selenium.webdriver.remote.webelement.WebElement (session="21c865ccacb0bf97542a8c3bab67d995", element="f.71720905CC681D8A2C7C6109A061CFE8.d.6FDE53B3DA5B09DDFD6AF65E28021CAC.e.394")>

In [27]:
while True:
    try:
        show_more_elm.click()
        sleep(2)
    except:
        print('We are at the end of the page!')
        break

We are at the end of the page!


### Lấy danh sách sản phẩm

In [33]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
len(product_list)

274

#### Tạo Func lấy thông 1 tin sản phẩm

In [34]:
def get_product_info(product_elm):
    phone_name = product_elm.find_element(By.XPATH, './/h3').text

    try:
        phone_price = product_elm.find_element(By.XPATH, './/strong[@class="price"]').text
    except:
        phone_price = "N/A"

    
    if phone_price == "N/A":
        try:
            phone_price = product_elm.find_element(By.CLASS_NAME, 'box-price-present').text
        except:
            phone_price = "N/A"
        
    result = {
        'name': phone_name,
        'price': phone_price
    }
    return result

In [ ]:
# Hàm lấy review của tất cả điện thoại
all_reviews = []
tmp = 0
def getReviewAllPhone(product_list):
    for item in product_list:
        product_info = get_product_info(item)
        item.click()
        sleep(2)

        try:
            # Nếu mà có readmore thì mình bấm luôn
            # read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[11]/div[2]/div/div/div[6]/div/a')
            read_more_elm = driver.find_element(By.CLASS_NAME, 'btn-view-all')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        try:
                            all_reviews.append({
                                'product_name': product_info['name'],
                                'product_price': product_info['price'],
                                'review': review.text
                            })
                        except:
                            continue

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None # Init var
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page: # if next_page exist
                            next_page.click() # forward
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            # Lấy xong hết review của trang cuối cùng của sản phẩm
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                try:
                                    all_reviews.append({
                                        'product_name': product_info['name'],
                                        'product_price': product_info['price'],
                                        'review': review.text
                                    })
                                except:
                                    continue
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:

            # Nếu có review nhưng chỉ có ===>> 1 TRANG
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            # Lấy xong thì back về
            driver.back()
            sleep(2)
            listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
            product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'ajaxed')

In [39]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

all_reviews = []
tmp = 0

def getReviewAllPhone_v02(product_list):
    global tmp
    i = 64  # Bắt đầu từ 64
    while i < len(product_list):
        # Lấy thông tin sản phẩm và click vào sản phẩm
        product_info = get_product_info(product_list[i])
        item = product_list[i]
        item.click()
        sleep(2)

        try:
            # Nếu có nút "Xem thêm" thì bấm
            read_more_elm = driver.find_element(By.CLASS_NAME, 'btn-view-all')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        try:
                            all_reviews.append({
                                'product_name': product_info['name'],
                                'product_price': product_info['price'],
                                'review': review.text
                            })
                        except:
                            continue

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page:
                            next_page.click()
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                try:
                                    all_reviews.append({
                                        'product_name': product_info['name'],
                                        'product_price': product_info['price'],
                                        'review': review.text
                                    })
                                except:
                                    continue
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:
            # Nếu chỉ có 1 trang review
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            driver.back()
            sleep(2)

        # Sau khi quay lại, làm mới product_list
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'listproduct'))
        )
        listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
        product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
        print(f"Độ dài product_list sau khi làm mới: {len(product_list)}")
        
        i += 1  # Tăng i để xử lý sản phẩm tiếp theo

In [40]:
getReviewAllPhone_v02(product_list)

Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_list sau khi làm mới: 274
Độ dài product_li

In [15]:
getReviewAllPhone(product_list)

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=134.0.6998.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CABBC085+28581]
	(No symbol) [0x00007FF7CAB243C0]
	(No symbol) [0x00007FF7CA9B91DA]
	(No symbol) [0x00007FF7CA9CF2E4]
	(No symbol) [0x00007FF7CA9CDDB3]
	(No symbol) [0x00007FF7CA9C1851]
	(No symbol) [0x00007FF7CA9BF632]
	(No symbol) [0x00007FF7CA9C38EC]
	(No symbol) [0x00007FF7CA9C39BF]
	(No symbol) [0x00007FF7CAA0F050]
	(No symbol) [0x00007FF7CAA0F9CC]
	(No symbol) [0x00007FF7CAA0213C]
	(No symbol) [0x00007FF7CAA3797F]
	(No symbol) [0x00007FF7CAA02006]
	(No symbol) [0x00007FF7CAA37B50]
	(No symbol) [0x00007FF7CAA60193]
	(No symbol) [0x00007FF7CAA37713]
	(No symbol) [0x00007FF7CAA00400]
	(No symbol) [0x00007FF7CAA016B3]
	GetHandleVerifier [0x00007FF7CAF12D2D+3529805

In [41]:
all_reviews

[{'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'Rất ok nhéNgắn gọn tiện lợi'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'gọn nhẹ dễ sử dụng'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'Tốt'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'Dùng tốt .Các bạn nhân vien tuyệt vời'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'Ok'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'Chất lượng tốt, nhân viên tiếp thị tuyệt vời'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'Bình thường'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'review': 'Sản phẩm tốt'},
 {'product_name': 'Cáp Type C 0.2m AVA+ DS06C',
  'product_price': '50.000₫',
  'revie

In [42]:
df = pd.DataFrame(all_reviews)
# Lưu file
df.to_csv("E:\\study\\tu_hoc\\DE\\CRAWL_DATA\\review_data\\data\\charger_data_64_continue.xlsx", index=False)